In [1]:
# Snowpark for Python
from snowflake.snowpark.session import Session
#from snowflake.snowpark.functions import month,year,col,sum
#from snowflake.snowpark.version import VERSION
# Misc
import json
import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)

In [2]:
param = json.load(open('connection_movie.json'))

In [3]:
session = Session.builder.configs(param).create()

In [4]:
session

In [5]:
session.sql_simplifier_enabled = True

In [6]:
snowflake_environment = session.sql('select current_user(), current_version()').collect()

In [6]:
session.get_current_role()

'"ACCOUNTADMIN"'

In [7]:
session.get_current_database(), session.get_current_schema(), session.get_current_warehouse()

('"MOVIE_DB"', '"MOVIE_SCHEMA"', '"MOVIE_L"')

In [8]:
tables = session.sql('show tables').collect()

In [9]:
tables[0]['name']

'ITEMS'

In [10]:
for table in tables:
    print(table['name'].lower())

items
ratings
users


In [11]:
dict_of_vars = vars()

In [12]:
# dict_of_vars = vars()
# tables from DB to data frames
list_of_table_names = []
for table in tables:
    table_name = table['name'].lower()
    list_of_table_names.append(table_name)
    # converts strings items, users, retings into variables with the same name
    # saving a data frame object
    dict_of_vars[table_name] = session.table(table_name)

In [13]:
list_of_table_names

['items', 'ratings', 'users']

In [14]:
items.show(2)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ITEM_ID"  |"TITLE"           |"RELEASE_DATE"  |"IMDB_URL"                                          |"UNKNOWN_GENRE"  |"ACTION"  |"ADVENTURE"  |"ANIMATION"  |"CHILDREN"  |"COMEDY"  |"CRIME"  |"DOCUMENTARY"  |"DRAMA"  |"FANTASY"  |"FILM_NOIR"  |"HORROR"  |"MUSICAL"  |"MYSTERY"  |"ROMANCE"  |"SCI_FI"  |"THRILLER"  |"WAR"  |"WESTERN"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
users.show(2)

-------------------------------------------------------
|"USER_ID"  |"AGE"  |"GENDER"  |"OCCUPATION"  |"ZIP"  |
-------------------------------------------------------
|1          |24     |M         |technician    |85711  |
|2          |53     |F         |other         |94043  |
-------------------------------------------------------



In [16]:
ratings.show(2)

--------------------------------------------------
|"USER_ID"  |"ITEM_ID"  |"RATING"  |"TIMESTAMP"  |
--------------------------------------------------
|196        |242        |3.0       |881250949    |
|186        |302        |3.0       |891717742    |
--------------------------------------------------



In [17]:
items.queries

{'queries': ['SELECT  *  FROM (items)'], 'post_actions': []}

In [18]:
items.explain()

---------DATAFRAME EXECUTION PLAN----------
Query List:
1.
SELECT  *  FROM (items)
Logical Execution Plan:
GlobalStats:
    partitionsTotal=1
    partitionsAssigned=1
    bytesAssigned=73728
Operations:
1:0     ->Result  ITEMS.ITEM_ID, ITEMS.TITLE, ITEMS.RELEASE_DATE, ITEMS.IMDB_URL, ITEMS.UNKNOWN_GENRE, ITEMS.ACTION, ITEMS.ADVENTURE, ITEMS.ANIMATION, ITEMS.CHILDREN, ITEMS.COMEDY, ITEMS.CRIME, ITEMS.DOCUMENTARY, ITEMS.DRAMA, ITEMS.FANTASY, ITEMS.FILM_NOIR, ITEMS.HORROR, ITEMS.MUSICAL, ITEMS.MYSTERY, ITEMS.ROMANCE, ITEMS.SCI_FI, ITEMS.THRILLER, ITEMS.WAR, ITEMS.WESTERN  
1:1          ->TableScan  MOVIE_DB.MOVIE_SCHEMA.ITEMS  ITEM_ID, TITLE, RELEASE_DATE, IMDB_URL, UNKNOWN_GENRE, ACTION, ADVENTURE, ANIMATION, CHILDREN, COMEDY, CRIME, DOCUMENTARY, DRAMA, FANTASY, FILM_NOIR, HORROR, MUSICAL, MYSTERY, ROMANCE, SCI_FI, THRILLER, WAR, WESTERN  {partitionsTotal=1, partitionsAssigned=1, bytesAssigned=73728}

--------------------------------------------


In [20]:
items_df = items.to_pandas()

In [21]:
ratings_df = ratings.to_pandas()
users_df = users.to_pandas()

In [24]:
users_df.head(2)

,USER_ID,AGE,GENDER,OCCUPATION,ZIP
0,1,24,M,technician,85711
1,2,53,F,other,94043


In [25]:
ratings_df.head(2)

,USER_ID,ITEM_ID,RATING,TIMESTAMP
0,196,242,3.0,881250949
1,186,302,3.0,891717742


In [26]:
items_df.head(2)

,ITEM_ID,TITLE,RELEASE_DATE,IMDB_URL,UNKNOWN_GENRE,ACTION,ADVENTURE,ANIMATION,CHILDREN,COMEDY,...,FANTASY,FILM_NOIR,HORROR,MUSICAL,MYSTERY,ROMANCE,SCI_FI,THRILLER,WAR,WESTERN
0,1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%2...,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
1,2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(...,False,True,True,False,False,False,...,False,False,False,False,False,False,False,True,False,False
